## Segmenting and Clustering of Toronto Neighborhoods Part 1

### Load relevant packages

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import requests

!conda install -c conda-forge BeautifulSoup4 --yes
from bs4 import BeautifulSoup

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

### Parse the HTML content using BeautifulSoup

In [2]:
# Use beautiful soup to parse the html content
postal_codes_page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').content
soup = BeautifulSoup(postal_codes_page,'html.parser')

### Define functions to extract the header and rows from the HTML table

In [27]:
def get_header(table):
    headers = []
    for th in table.find("tr").find_all("th"):
        headers.append(th.text.strip())
    return headers

def get_rows(table):
    rows = []
    for tr in table.find_all("tr")[1:]:
        row_entry = []
        tds = tr.find_all("td")
        
        for td in tds:
            row_entry.append(td.text.strip())
            
        rows.append(row_entry)
    return rows

### Create Pandas dateframe from the lists of rows and columns 

In [93]:
postal_codes_table = soup.find('table')
headers = get_header(postal_codes_table)
rows = get_rows(postal_codes_table)
df = pd.DataFrame(rows, columns=headers)

df.head(5)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Drop all the rows that have no borough assigned to them

In [94]:
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)

df.head(5)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Combine rows that have the same postal code

In [95]:
df = df.groupby(['Postal Code','Borough'], sort=False).agg({'Neighborhood': ', '.join,}).reset_index()

df.head(5)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Replace all the rows that have a missing neighborhood with the borough

In [96]:
df.loc[(df['Neighborhood'] == 'Not assigned'),'Neighborhood'] = df['Borough']

df.head(5)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Save the dataframe as a CSV file for use in the rest of the project

In [98]:
df.to_csv('toronto-codes.csv')

### Get the shape of the table

In [99]:
df.shape

(103, 3)